# Pandas Dataframe 의 다양한 iteration 방법을 비교합니다.

- requirements: `pandas >= 1.0` 
- task:
    - text의 문장을 token length 기준 100까지 자르기


1. `iterrows`
2. `loc`, `iloc`
3. `at`, `iat`
4. `values`

In [6]:
import pandas as pd

In [7]:
data = pd.read_csv('../sample_data/wikipedia_split_psgs_w100.tsv', sep='\t')

In [3]:
data.head()

,id,text,title
0,1,"지미 카터 제임스 얼 ""지미"" 카터 주니어(, 1924년 10월 1일 ~ )는 민주...",지미 카터
1,2,등용법을 내세웠다. 1976년 미합중국 (미국) 제39대 대통령 선거에 민주당 후보...,지미 카터
2,3,구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 ...,지미 카터
3,4,"그는 이에 대해 애매한 태도를 보였고, 이는 후에 대한민국 내에서 고조되는 반미 운...",지미 카터
4,5,"카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국...",지미 카터


In [4]:
def cut_text(text, max_len: int = 100):
    return ' '.join(text.split()[:max_len])

### 1. `iterrows`

In [23]:
%%timeit

result = []
for i, row in data.iterrows():
    short_text = cut_text(row['text'])
    instance = {
        'id': row['id'],
        'text': short_text,
        'title': row['title']
    }
    result.append(instance)

62.7 ms ± 729 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. `loc`, `iloc`

In [26]:
%%timeit

result = []
for idx in data.index:
    short_text = cut_text(data.loc[idx, 'text'])
    instance = {
        'id': data.loc[idx, 'id'],
        'text': short_text,
        'title': data.loc[idx, 'title']
    }
    result.append(instance)

24.6 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### 주의: `loc`을 잘못 쓰면 시간이 더 길어진다

In [8]:
%%timeit

result = []
for idx in data.index:
    short_text = cut_text(data.loc[idx]['text'])  # diff
    instance = {
        'id': data.loc[idx]['id'],
        'text': short_text,
        'title': data.loc[idx]['title']
    }
    result.append(instance)

268 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit

result = []
for idx in data.index:
    row = data.loc[idx]
    short_text = cut_text(row['text'])  # diff
    instance = {
        'id': row['id'],
        'text': short_text,
        'title': row['title']
    }
    result.append(instance)

99.4 ms ± 904 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3. `at`, `iat`

In [30]:
%%timeit

result = []
for idx in data.index:
    short_text = cut_text(data.at[idx, 'text'])
    instance = {
        'id': data.at[idx, 'id'],
        'text': short_text,
        'title': data.at[idx, 'title']
    }
    result.append(instance)

15.8 ms ± 49.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 4. `itertuples`

In [35]:
%%timeit

result = []
for row in data.itertuples():
    short_text = cut_text(row.text)
    instance = {
        'id': row.id,
        'text': short_text,
        'title': row.title
    }
    result.append(instance)

7.7 ms ± 21.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 5. `values`

단, `values`는 column 값으로 인자를 받아올 수 없다는 단점이 있다. `text`에 해당하는 인자를 받아오기 위해서는 `text` 가 원 데이터에서 몇 번째 column 이었는지를 알아야 한다

In [31]:
%%timeit

result = []
for value in data.values:
    short_text = cut_text(value[1])
    instance = {
        'id': value[0],
        'text': short_text,
        'title': value[2]
    }
    result.append(instance)

7.1 ms ± 43.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



### 6. `apply`

In [35]:
%%timeit

result = data.copy()
result['text'] = result['text'].apply(lambda x: cut_text(x))
result.to_dict(orient='records')

9.91 ms ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
